In [1]:
import sys

## find 'core' package
sys.path.insert(0, '../..')

sys.version

'3.7.3 (default, Mar 27 2019, 09:23:15) \n[Clang 10.0.1 (clang-1001.0.46.3)]'

In [2]:
from core.models import unit

from core.models.table import Table
from core.factories import column_factory

from core.orchestrators import ManyToManySameInputAndOutputVocabOrchestrator

In [3]:
vocab = 'abcdefghijklmnopqrstuvwxyz'

vocab_orchestrator = ManyToManySameInputAndOutputVocabOrchestrator(vocab, unit.get_unit_lookup())

In [4]:
table_1 = Table()
table_1.set_id('0')
table_1.n_spaces_between_columns = 5
table_1.max_number_of_labels = 3
table_1.columns = [
    column_factory.create_column(
        order = 0,
        min_label_length = 2,
        max_label_length = 12,
        max_unit_length = 10,
        max_number_of_label_parts = 2
    ),
    column_factory.create_column(
        order = 1,
        min_label_length = 2,
        max_label_length = 12,
        max_unit_length = 10,
        max_number_of_label_parts = 2
    )
]

In [5]:
table_lookup = {}

table_lookup[table_1.get_id()] = table_1

In [6]:
n_instances = 300

options = []
for key in table_lookup.keys():
    options.append(
        {
            'table_generator': vocab_orchestrator.create_table_generator(table_1),
            'n_instances': n_instances
        }
    )
    
dataset = vocab_orchestrator.build_dataset(options)
    
expected_size = len(table_lookup.keys()) * n_instances
assert len(dataset) == expected_size, f'{len(dataset)} <> {expected_size}'

dataset.head(n=5)

,table_id,table,target
0,0,fbi 1.18 cm/s cbdxt 1.55 m/...,fbi 1.18 cm/s\nhbqqbrlflvy 3.03 cm\ncbdxt 1.55...
1,0,jgjk 0.82 m/s kgxxe 2.73 cm...,jgjk 0.82 m/s\nhbqqbrlflvy 0.34 cm\nkgxxe 2.73...
2,0,jgjk 2.14 m/s kgxxe 0.77 cm...,jgjk 2.14 m/s\nhbqqbrlflvy 3.38 cm\nkgxxe 0.77...
3,0,fbi 1.15 cm/s dl 1.54 cm...,fbi 1.15 cm/s\nhbqqbrlflvy 3.92 cm\ndl 1.54 cm...
4,0,jgjk 2.05 m/s dl 4.25 cm...,jgjk 2.05 m/s\nhbqqbrlflvy 3.02 cm\ndl 4.25 cm


In [7]:
## breakdown,
dataset.groupby('table_id').count()

,table,target
table_id,,
0,300,300


In [8]:
## save off,
dataset.to_csv('../../data/table/v0.0.1_actual.csv', index = False)

In [9]:
translated_df = vocab_orchestrator.translate_to_integer_array(dataset, table_lookup)

translated_df.head()

,input,output,n_columns
0,"[21, 17, 24, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 2, ...","[21, 17, 24, 1, 5, 2, 5, 12, 1, 18, 28, 3, 34,...",2
1,"[25, 22, 25, 26, 1, 1, 1, 1, 1, 1, 1, 1, 4, 2,...","[25, 22, 25, 26, 1, 4, 2, 12, 6, 1, 28, 3, 34,...",2
2,"[25, 22, 25, 26, 1, 1, 1, 1, 1, 1, 1, 1, 6, 2,...","[25, 22, 25, 26, 1, 6, 2, 5, 8, 1, 28, 3, 34, ...",2
3,"[21, 17, 24, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 2, ...","[21, 17, 24, 1, 5, 2, 5, 9, 1, 18, 28, 3, 34, ...",2
4,"[25, 22, 25, 26, 1, 1, 1, 1, 1, 1, 1, 1, 6, 2,...","[25, 22, 25, 26, 1, 6, 2, 4, 9, 1, 28, 3, 34, ...",2


In [10]:
## save off,
translated_df.to_csv('../../data/table/v0.0.1.csv', index = False)